# Generate Events

### Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging
import numpy as np
import matplotlib
import math
from matplotlib import pyplot as plt

from madminer.core import MadMiner
from madminer.delphes import DelphesReader
from madminer.sampling import combine_and_shuffle
from madminer.plotting import plot_distributions


# MadMiner output
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

## Madgraph

### Signal

We use Madgraph to generate events at all benchmark points.

Due to the large number of benchmark points, the file sizes get pretty ridiculous. For ~120 benchmark points, you will need roughly 120 GB of space in the mg_process_directory. 

This cell takes ~2 days to complete on my computer (I get about 3 benchmark points / hr). Keep in mind that the evaluation time scales with the number of benchmark points squared, as when the number of benchmark points is large, the evaluation of joint likelihood ratios at each benchmark quickly takes the majority of the computation time.

In [10]:
# Be sure to change these to the directory of the Madgraph installation.
###########################################################################################
mg_dir = '/home/kwl/Documents/hep/MG5_aMC_v2_6_5'    # Points to Madgraph Install     
events_dir = '/udd/madminer_events/'            # Points to directory to save events 
                                                     # (where the run_XX folders are saved)
###########################################################################################
miner = MadMiner()
miner.load("meta/setup.h5")

miner.run_multiple(
    sample_benchmarks=None,
    mg_directory=mg_dir,
    mg_process_directory=events_dir+'signal_pythia',
    proc_card_file='cards/proc_card.dat',
    param_card_template_file='cards/param_card.dat',
    pythia8_card_file='cards/pythia8_card.dat',
    run_card_files=['cards/run_card_signal.dat'],
    log_directory='logs/signal'
)


00:46 madminer.core        INFO    Found 2 parameters:
00:46 madminer.core        INFO       mzp (LHA: mass 56, maximal power in squared ME: (0,), range: (700.0, 800.0))
00:46 madminer.core        INFO       gzp (LHA: dminputs 2, maximal power in squared ME: (0,), range: (0.0, 2.0))
00:46 madminer.core        INFO    Found 121 benchmarks:
00:46 madminer.core        INFO       benchmark_0: mzp = 7.00e+02, gzp = 0.00e+00
00:46 madminer.core        INFO       benchmark_1: mzp = 7.00e+02, gzp = 0.20
00:46 madminer.core        INFO       benchmark_2: mzp = 7.00e+02, gzp = 0.40
00:46 madminer.core        INFO       benchmark_3: mzp = 7.00e+02, gzp = 0.60
00:46 madminer.core        INFO       benchmark_4: mzp = 7.00e+02, gzp = 0.80
00:46 madminer.core        INFO       benchmark_5: mzp = 7.00e+02, gzp = 1.00
00:46 madminer.core        INFO       benchmark_6: mzp = 7.00e+02, gzp = 1.20
00:46 madminer.core        INFO       benchmark_7: mzp = 7.00e+02, gzp = 1.40
00:46 madminer.core        INFO

00:46 madminer.core        INFO       benchmark_100: mzp = 7.90e+02, gzp = 0.20
00:46 madminer.core        INFO       benchmark_101: mzp = 7.90e+02, gzp = 0.40
00:46 madminer.core        INFO       benchmark_102: mzp = 7.90e+02, gzp = 0.60
00:46 madminer.core        INFO       benchmark_103: mzp = 7.90e+02, gzp = 0.80
00:46 madminer.core        INFO       benchmark_104: mzp = 7.90e+02, gzp = 1.00
00:46 madminer.core        INFO       benchmark_105: mzp = 7.90e+02, gzp = 1.20
00:46 madminer.core        INFO       benchmark_106: mzp = 7.90e+02, gzp = 1.40
00:46 madminer.core        INFO       benchmark_107: mzp = 7.90e+02, gzp = 1.60
00:46 madminer.core        INFO       benchmark_108: mzp = 7.90e+02, gzp = 1.80
00:46 madminer.core        INFO       benchmark_109: mzp = 7.90e+02, gzp = 2.00
00:46 madminer.core        INFO       benchmark_110: mzp = 8.00e+02, gzp = 0.00e+00
00:46 madminer.core        INFO       benchmark_111: mzp = 8.00e+02, gzp = 0.20
00:46 madminer.core        INFO     

## Delphes

### Stage events

In [11]:
delphes = DelphesReader('meta/setup.h5')

for i in range(len(miner.benchmarks.keys())):
    run_name=str(i+1)
    if i < 9:
        run_name = '0' + str(i+1)
    delphes.add_sample(
        lhe_filename=events_dir + 'signal_pythia/Events/run_' + run_name + '/unweighted_events.lhe.gz',
        hepmc_filename=events_dir + 'signal_pythia/Events/run_' + run_name + '/tag_1_pythia8_events.hepmc.gz',
        sampled_from_benchmark=miner.benchmarks.keys()[i],
        is_background=False
    )



### Run Detector

In [12]:
delphes.run_delphes(
    delphes_directory=mg_dir + '/Delphes',
    delphes_card='cards/delphes_card.dat',
    log_file='logs/delphes.log',
)

01:19 madminer.delphes     INFO    Running Delphes on HepMC sample at /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_49/tag_1_pythia8_events.hepmc.gz


If you want to load events from a root file (you already ran Delphes) run the cell below instead of staging + detector.

In [17]:
delphes = DelphesReader('meta/setup_ppjj_largem.h5')
# Need 44-49
for i in range(len(miner.benchmarks.keys())):
    run_name=str(i+1)
    if i < 9:
        run_name = '0' + str(i+1)
    delphes.add_sample(
        lhe_filename=events_dir + 'signal_pythia/Events/run_' + run_name + '/unweighted_events.lhe.gz',
        hepmc_filename=events_dir + 'signal_pythia/Events/run_' + run_name + '/tag_1_pythia8_events.hepmc.gz',
        delphes_filename = events_dir + 'signal_pythia/Events/run_' + run_name + '/tag_1_pythia8_events_delphes.root',
        sampled_from_benchmark=miner.benchmarks.keys()[i],
        is_background=False
    )

u"\ndelphes.add_sample(\n    lhe_filename=events_dir + 'background_pythia/Events/run_01/unweighted_events.lhe.gz',\n    hepmc_filename=events_dir + 'background_pythia/Events/run_01/tag_1_pythia8_events.hepmc.gz',\n    delphes_filename = events_dir + 'background_pythia/Events/run_01/tag_1_pythia8_events_delphes.root',\n    sampled_from_benchmark='benchmark_0',\n    is_background=True\n)\n"

### Save detector level data

We have 8 features, which are the 2 leading jet four-momenta:

leading jet: $[\eta_0, \phi_0, p_{t,0}, m_0]$

subleading jet: $[\eta_1, \phi_1, p_{t,1}, m_1]$

We cut the invariant mass so that $m_{jj}=\sqrt{2 |p_{t,0}| |p_{t,1}| \left(\cosh(\eta_0 - \eta_1) - \cos(\phi_0 - \phi_1)\right)}\in[200, 400]~$GeV

*Make sure that you are currently running root with source xx/xxx/rootdir/bin/thisroot.sh, or however you run root*

In [20]:
delphes.reset_cuts()

delphes.add_observable(
    'eta0',
    'j[0].eta',
    required=True
)
delphes.add_observable(
    'phi0',
    'j[0].phi()',
    required=True,
)
delphes.add_observable(
    'pt0',
    'j[0].pt',
    required=True,
)
delphes.add_observable(
    'm0',
    'j[0].m',
    required=True,
)
delphes.add_observable(
    'eta1',
    'j[1].eta',
    required=True,
)
delphes.add_observable(
    'phi1',
    'j[1].phi()',
    required=True,
)
delphes.add_observable(
    'pt1',
    'j[1].pt',
    required=True,
)
delphes.add_observable(
    'm1',
    'j[1].m',
    required=True,
)

#### Cuts ####
invariant_mass = 'sqrt(2*abs(j[0].pt)*abs(j[1].pt)*(cosh(j[0].eta-j[1].eta)-cos(j[0].phi()-j[1].phi())))'
delphes.add_cut(invariant_mass + '>150')
delphes.add_cut(invariant_mass + '<450')

#### Process & Export ####

delphes.analyse_delphes_samples()
delphes.save('meta/delphes_data.h5')
combine_and_shuffle(
    ['meta/delphes_data.h5'],
    'meta/delphes_data_shuffled.h5'
)

15:17 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_01/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:18 madminer.utils.inter INFO      710 / 20000 events pass everything
15:18 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_02/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:18 madminer.utils.inter INFO      706 / 20000 events pass everything
15:18 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_03/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:18 madminer.utils.inter INFO      655 / 20000 events pass everything
15:18 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_04/tag_1_pythia8_events_delphes.root

15:28 madminer.utils.inter INFO      1223 / 20000 events pass everything
15:28 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_31/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:28 madminer.utils.inter INFO      1526 / 20000 events pass everything
15:28 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_32/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:28 madminer.utils.inter INFO      1988 / 20000 events pass everything
15:29 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_33/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:29 madminer.utils.inter INFO      2586 / 20000 events pass everything
15:29 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_even

15:38 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_60/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:38 madminer.utils.inter INFO      710 / 20000 events pass everything
15:38 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_61/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:38 madminer.utils.inter INFO      830 / 20000 events pass everything
15:39 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_62/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:39 madminer.utils.inter INFO      977 / 20000 events pass everything
15:39 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_63/tag_1_pythia8_events_delphes.root

15:48 madminer.utils.inter INFO      671 / 20000 events pass everything
15:49 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_90/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:49 madminer.utils.inter INFO      650 / 20000 events pass everything
15:49 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_91/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:49 madminer.utils.inter INFO      684 / 20000 events pass everything
15:49 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_92/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:49 madminer.utils.inter INFO      701 / 20000 events pass everything
15:50 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_p

15:58 madminer.utils.inter INFO      998 / 20000 events pass everything
15:59 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_119/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:59 madminer.utils.inter INFO      1221 / 20000 events pass everything
15:59 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_120/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
15:59 madminer.utils.inter INFO      1605 / 20000 events pass everything
15:59 madminer.delphes     INFO    Analysing Delphes sample /udd/madminer_events_ppjj_largem/signal_pythia/Events/run_121/tag_1_pythia8_events_delphes.root: Calculating 8 observables, requiring 3 selection cuts
16:00 madminer.utils.inter INFO      2028 / 20000 events pass everything
16:00 madminer.delphes     INFO    Analysed number of events per sampling be

16:00 madminer.sampling    INFO    Copying setup from meta/delphes_data_ppjj.h5 to meta/delphes_data_shuffled_ppjj.h5
16:00 madminer.sampling    INFO    Loading samples from file 1 / 1 at meta/delphes_data_ppjj.h5, multiplying weights with k factor 1.0
